In [1]:
pip install dash sqlalchemy pymysql pandas plotly

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sqlalchemy import create_engine

##DB Credentials
db_user='root'
db_password='admin'
db_host='localhost'
db_name='airport_db'

##Create Connection Engine
engine= create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

## Query Data
query= "SELECT * from airports2"
df= pd.read_sql(query,engine)
df.head()

,Origin_airport,Destination_airport,Origin_city,Destination_city,Passengers,Seats,Flights,Distance,Fly_date,Origin_population,Destination_population,Org_airport_lat,Org_airport_long,Dest_airport_lat,Dest_airport_long
0,MHK,AMW,"Manhattan, KS","Ames, IA",21,30,1,254,2008-10-01,122049,86219,39.140999,-96.670799,NA,NA
1,EUG,RDM,"Eugene, OR","Bend, OR",41,396,22,103,1990-11-01,284093,76034,44.124599,-123.211998,44.2541008,-121.1500015
2,EUG,RDM,"Eugene, OR","Bend, OR",88,342,19,103,1990-12-01,284093,76034,44.124599,-123.211998,44.2541008,-121.1500015
3,EUG,RDM,"Eugene, OR","Bend, OR",11,72,4,103,1990-10-01,284093,76034,44.124599,-123.211998,44.2541008,-121.1500015
4,MFR,RDM,"Medford, OR","Bend, OR",0,18,1,156,1990-02-01,147300,76034,42.374199,-122.873001,44.2541008,-121.1500015


In [7]:
## Create Simple Dash Application
from dash import Dash, dcc, html
import plotly.express as px

##Create Dash App
app= Dash(__name__)

##Plot using plotly (Passengers data over distance)
fig = px.scatter(df, x='Distance', y='Passengers', color='Origin_airport',
                 title='Passengers Vs Distance', labels={'Passengers':'Passengers','Distance':'Distance (Miles)'})

fig_bar = px.bar(df, x='Origin_airport',y='Seats',color='Destination_airport',
                 title='Seats per flight', labels={'Seats':'No of Seats','Origin_airport':'Origin Airport'})

app.layout = html.Div(children=[
    html.H1(children='Flight Dashboard'),
    dcc.Graph(
        id='passenger-distance-graph',
        figure=fig
    ),
    dcc.Graph(
        id='seats-bar-chart',
        figure=fig_bar
    ),
    html.Div(children='Dashboard for visualization of flight data')
])                 

if __name__=='__main__':
    app.run_server(debug=True)

In [20]:
from dash import dcc, html, Input, Output
import dash
import plotly.express as px
import pandas as pd

# Reuse the same data (df) from the previous code

# Create the Dash App
app = dash.Dash(__name__)

# Layout with Dropdown and DatePicker
app.layout = html.Div([
    html.H1("Flight Dashboard"),

    # Dropdown for selecting the Origin Airport
    dcc.Dropdown(
        id='origin-airport-dropdown',
        options=[{'label': airport, 'value': airport} for airport in df['Origin_airport'].unique()],
        placeholder="Select an Origin Airport",
        multi=True
    ),

    # Date range picker for Fly_date
    dcc.DatePickerRange(
        id='date-picker-range',
        start_date=df['Fly_date'].min(),
        end_date=df['Fly_date'].max(),
        display_format='YYYY-MM-DD'
    ),

    # Graph
    dcc.Graph(id='filtered-passenger-distance-graph'),
    
    dcc.Graph(id='filtered-seats-bar-chart')
])

# Callback for interactivity
@app.callback(
    [Output('filtered-passenger-distance-graph', 'figure'),
     Output('filtered-seats-bar-chart', 'figure')],
    [Input('origin-airport-dropdown', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_graphs(selected_airports, start_date, end_date):
    # Filter data based on selected airports and date range
    filtered_df = df[(df['Fly_date'] >= start_date) & (df['Fly_date'] <= end_date)]
    if selected_airports:
        filtered_df = filtered_df[filtered_df['Origin_airport'].isin(selected_airports)]
    
    # Update scatter plot (Passengers vs Distance)
    scatter_fig = px.scatter(filtered_df, x='Distance', y='Passengers', color='Origin_airport',
                             title="Filtered Passengers vs Distance")

    # Update bar chart (Seats per Origin Airport)
    bar_fig = px.bar(filtered_df, x='Origin_airport', y='Seats', color='Destination_airport',
                     title="Filtered Seats per Flight")

    return scatter_fig, bar_fig

if __name__ == '__main__':
    app.run_server(debug=True)
